### Integração Numérica - Regra do Trapézio

Este notebook apresenta a implementação da Regra do Trapézio para cálculo de integrais definidas, incluindo o erro de aproximação simples e o erro generalizado.

In [1]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math
import numpy as np
import sympy as sp

### Classe `IntegralTrapezio`

In [2]:
class IntegralTrapezio(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None,
                 func_integral=None, h=None, erro_desejado=None):
        super().__init__()

        if erro_desejado is not None:
            if limite_inf is None or limite_sup is None or func_integral is None:
                raise ValueError("Para usar erro_desejado, forneça limite_inf, limite_sup e func_integral.")

            self.limite_inf = limite_inf
            self.limite_sup = limite_sup
            self.func_expr = func_integral  # guarda a expressão simbólica original

            # Converte expressão simbólica para função numérica
            x_sym = sp.Symbol('x')
            func_lambd = sp.lambdify(x_sym, func_integral, modules=["math"])

            # Tenta obter a segunda derivada simbólica
            K = self._derivada_segunda_simbolica(func_integral, limite_inf, limite_sup)

            if K is None:
                raise ValueError("Não foi possível calcular a segunda derivada simbolicamente.")

            n_quadrado_min = (K * (limite_sup - limite_inf) ** 3) / (12 * erro_desejado)
            n = math.ceil(math.sqrt(n_quadrado_min))
            n = min(max(n, 4), 10000)
            print(f"valor de n (subintervalos) encontrado: {n}")
            h = (limite_sup - limite_inf) / n
            print(f"valor de h calculado: {h}")

            self._setup(x, y, limite_inf, limite_sup, func_lambd, h)

        else:
            self._setup(x, y, limite_inf, limite_sup, func_integral, h)

    def _derivada_segunda_simbolica(self, func_expr, a, b):
        x = sp.Symbol('x')
        try:
            f_diff2 = sp.diff(func_expr, x, 2)
            f_diff2_func = sp.lambdify(x, f_diff2, modules=["math"])
            pontos = [a, (a + b) / 2, b]
            return max(abs(f_diff2_func(xi)) for xi in pontos)
        except Exception as e:
            print(f"[sympy fallback] Derivada simbólica falhou: {e}")
            return None

    def integral(self):
        # regra do trapézio simples sobre todos os pontos
        soma = self.y[0] + self.y[-1] + 2 * np.sum(self.y[1:-1])
        return (self.h / 2) * soma

    def erro_simples(self):
        d2_inf = self.interpolador.derivative(self.limite_inf, 2)
        d2_sup = self.interpolador.derivative(self.limite_sup, 2)
        maior = self.saber_maior(d2_inf, d2_sup)
        parcela_1 = ((self.limite_sup - self.limite_inf) * (self.h**2)) / 12
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        # No trapézio geralmente erro é dado pela segunda derivada (2ª ordem)
        return self.erro_simples()

### Exemplo de uso com pontos

In [89]:
# Questão velocidade - lista
def f(x):
    return x * math.sqrt(x**2 + 1)

pontos_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pontos_y = [0, 50.4, 53.7, 67.5, 74.3, 84.6, 92.1, 98.3, 100, 105, 110]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")


Trapézio (x, y): 780.90000
Erro simples: 1237.71209
Erro generalizado: 1237.71209


In [86]:
# Questão margens do rio - lista

pontos_x = [30, 50]
pontos_y = [171.2, 95.3]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio parte 2 da M2 (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")

pontos_x = [30, 50]
pontos_y = [72.8, 51]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio parte 2 da M1 (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")

Trapézio parte 2 da M2 (x, y): 2665.00000
Erro simples: 0.00000
Erro generalizado: 0.00000
Trapézio parte 2 da M1 (x, y): 1238.00000
Erro simples: 0.00000
Erro generalizado: 0.00000


In [87]:
# Questão 2 revisão - integral math.sqrt(x) com erro desejado 0.001
x = sp.Symbol('x')
f_expr = sp.sqrt(x)

trap2 = IntegralTrapezio(limite_inf=1, limite_sup=4, func_integral=f_expr, erro_desejado=0.01)
print(f"Trapézio (x, y): {trap2.integral():.4f}")
print(f"Erro simples: {trap2.erro_simples():.4f}")
print(f"Erro generalizado: {trap2.erro_generalizado():.4f}")


valor de n (subintervalos) encontrado: 8
valor de h calculado: 0.375
Trapézio (x, y): 4.6637
Erro simples: 0.0086
Erro generalizado: 0.0086


### Exemplo de uso com função

In [88]:
# Questão 2: integral de 0 a 1 3*exp(-x) - Lista
def f(x):
    return 3 * math.exp(-x)

trap2 = IntegralTrapezio(limite_inf=0, limite_sup=1, func_integral=f, h=1/5)
print(f"Trapézio (x, y): {trap2.integral():.4f}")
print(f"Erro simples: {trap2.erro_simples():.4f}")
print(f"Erro generalizado: {trap2.erro_generalizado():.4f}")


Trapézio (x, y): 1.9027
Erro simples: 0.0100
Erro generalizado: 0.0100


### AV3

In [6]:
# Questão 1 AV3: integral de 0 a 1 para x * math.sqrt((x**2) + 1) utilizando n (numero de aplicações da regra) = 2, 10 e 50
def f(x):
    return x * math.sqrt((x**2) + 1)

limite_sup = 1
limite_inf = 0
n = 2
h = (limite_sup - limite_inf) / n

trap = IntegralTrapezio(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Trapézio (x, y): {trap.integral():.4f}")
print(f"Erro simples: {trap.erro_simples():.4f}")
print(f"Erro generalizado: {trap.erro_generalizado():.4f}")

n = 10
h = (limite_sup - limite_inf) / n

trap = IntegralTrapezio(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Trapézio (x, y): {trap.integral():.4f}")
print(f"Erro simples: {trap.erro_simples():.4f}")
print(f"Erro generalizado: {trap.erro_generalizado():.4f}")

n = 50
h = (limite_sup - limite_inf) / n

trap = IntegralTrapezio(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Trapézio (x, y): {trap.integral():.4f}")
print(f"Erro simples: {trap.erro_simples():.4f}")
print(f"Erro generalizado: {trap.erro_generalizado():.4f}")

para n = 2
Trapézio (x, y): 0.6331
Erro simples: 0.0247
Erro generalizado: 0.0247
para n = 10
Trapézio (x, y): 0.6104
Erro simples: 0.0015
Erro generalizado: 0.0015
para n = 50
Trapézio (x, y): 0.6095
Erro simples: 0.0000
Erro generalizado: 0.0000


c:\Users\019.705184\Documents\GitHub\NumericCalculations\IntegralMethods\integral.py:48: UserWarning: 51 degrees provided, degrees higher than about thirty cause problems with numerical instability with 'KroghInterpolator'
  self.interpolador = KroghInterpolator(self.x, self.y)


In [7]:
# Questão 4 AV3: integral de 2 a 4 para (ln(x) + x **2)/(x + 3)**2 com n = 8
def f(x):
    return (math.log(x) + x ** 2) / (x + 3) ** 2

limite_inf = 2
limite_sup = 4
n = 8
h = (limite_sup - limite_inf) / n

trap = IntegralTrapezio(limite_inf=limite_inf, limite_sup=limite_sup, func_integral=f, h=h)
print(f"Para n = {n}")
print(f"Trapézio (x, y): {trap.integral():.4f}")
print(f"Erro simples: {trap.erro_simples():.4f}")
print(f"Erro generalizado: {trap.erro_generalizado():.4f}")

Para n = 8
Trapézio (x, y): 0.5547
Erro simples: 0.0003
Erro generalizado: 0.0003
